In [1]:
ENV["LIBTENSORFLOW"] = "/etc/tensorflow/libtensorflow.so"

"/etc/tensorflow/libtensorflow.so"

In [2]:
using TensorFlow
using Test

In [3]:
sess = TensorFlow.Session()

x = TensorFlow.constant(Float64[1,2])
y = TensorFlow.Variable(Float64[3,4])
z = TensorFlow.placeholder(Float64)

w = exp(x + z + -y)

run(sess, TensorFlow.global_variables_initializer())
res = run(sess, w, Dict(z=>Float64[1,2]))
Test.@test res[1] ≈ exp(-1)

2018-09-14 21:16:36.189806: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2018-09-14 21:16:36.807269: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:964] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-09-14 21:16:36.807565: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1411] Found device 0 with properties: 
name: Tesla P4 major: 6 minor: 1 memoryClockRate(GHz): 1.1135
pciBusID: 0000:00:04.0
totalMemory: 7.43GiB freeMemory: 7.31GiB
2018-09-14 21:16:36.807584: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1490] Adding visible gpu devices: 0
2018-09-14 21:16:37.259822: I tensorflow/core/common_runtime/gpu/gpu_device.cc:971] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-09-14 21:16:37.259869: I tensorflow/core/common_runtime/gpu/gpu_device.cc:97

Test Passed

In [4]:
using TensorFlow
using Distributions
using Printf

In [5]:
# Generate some synthetic data
x = randn(100, 50)
w = randn(50, 10)
y_prob = exp.(x*w)
y_prob ./= sum(y_prob,dims=2)

function draw(probs)
    y = zeros(size(probs))
    for i in 1:size(probs, 1)
        idx = rand(Categorical(probs[i, :]))
        y[i, idx] = 1
    end
    return y
end

y = draw(y_prob)

# Build the model
sess = Session(Graph())

X = placeholder(Float64, shape=[-1, 50])
Y_obs = placeholder(Float64, shape=[-1, 10])

variable_scope("logisitic_model"; initializer=Normal(0, .001)) do
    global W = get_variable("W", [50, 10], Float64)
    global B = get_variable("B", [10], Float64)
end

Y=nn.softmax(X*W + B)

Loss = -reduce_sum(log(Y).*Y_obs)
optimizer = train.AdamOptimizer()
minimize_op = train.minimize(optimizer, Loss)
saver = train.Saver()

# Run training
run(sess, global_variables_initializer())
checkpoint_path = mktempdir()
@info("Checkpoint files saved in $checkpoint_path")
for epoch in 1:100
    cur_loss, _ = run(sess, [Loss, minimize_op], Dict(X=>x, Y_obs=>y))
    println(@sprintf("Current loss is %.2f.", cur_loss))
    train.save(saver, sess, joinpath(checkpoint_path, "logistic"), global_step=epoch)
end

2018-09-14 21:16:43.478016: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1490] Adding visible gpu devices: 0
2018-09-14 21:16:43.478078: I tensorflow/core/common_runtime/gpu/gpu_device.cc:971] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-09-14 21:16:43.478083: I tensorflow/core/common_runtime/gpu/gpu_device.cc:977]      0 
2018-09-14 21:16:43.478088: I tensorflow/core/common_runtime/gpu/gpu_device.cc:990] 0:   N 
2018-09-14 21:16:43.478196: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1103] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 7055 MB memory) -> physical GPU (device: 0, name: Tesla P4, pci bus id: 0000:00:04.0, compute capability: 6.1)
┌ Info: Checkpoint files saved in /tmp/tmpBPkO44
└ @ Main In[5]:39


Current loss is 230.30.
Current loss is 228.83.
Current loss is 227.38.
Current loss is 225.93.
Current loss is 224.49.
Current loss is 223.05.
Current loss is 221.63.
Current loss is 220.21.
Current loss is 218.80.
Current loss is 217.40.
Current loss is 216.01.
Current loss is 214.62.
Current loss is 213.24.
Current loss is 211.88.
Current loss is 210.52.
Current loss is 209.17.
Current loss is 207.83.
Current loss is 206.49.
Current loss is 205.17.
Current loss is 203.85.
Current loss is 202.55.
Current loss is 201.25.
Current loss is 199.96.
Current loss is 198.68.
Current loss is 197.41.
Current loss is 196.14.
Current loss is 194.89.
Current loss is 193.65.
Current loss is 192.41.
Current loss is 191.18.
Current loss is 189.96.
Current loss is 188.76.
Current loss is 187.56.
Current loss is 186.36.
Current loss is 185.18.
Current loss is 184.01.
Current loss is 182.84.
Current loss is 181.69.
Current loss is 180.54.
Current loss is 179.40.
Current loss is 178.27.
Current loss is 